# A 2-Hours Seminar about [InterCom](https://github.com/Tecnologias-multimedia/intercom)

## 1. What is InterCom
InterCom is a real-time intercommunicator (currently, only transmitting audio). The sequence of [frames](https://python-sounddevice.readthedocs.io/en/0.3.12/api.html) (two stereo samples) is splitted into chunks and each one is transmitted in an [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol) packet.

### A [minimal](https://tecnologias-multimedia.github.io/study_guide/minimal/) implementation

In [ ]:
# Interrupt with: Kernel -> Interrupt
!python ~/intercom/minimal.py --show_stats

## Theoretical bit-rate
$$
\frac{44100\frac{\text{frames}}{\text{second}}\times 2\frac{\text{samples}}{\text{frame}}\times 2\frac{\text{bytes}}{\text{sample}}\times 8\frac{\text{bits}}{\text{byte}}}{1000} = 1411.2 ~\text{kbps}
$$

## 2. De-jittering through [buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/)
Non-dedicated links use to have a significant [jitter](https://en.wikipedia.org/wiki/Jitter).
To hide the it, we can use buffering:

![Buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/graphics/timelines.svg)

### Which is the latency and jitter in my host?

In [ ]:
!ping -c 10 localhost

Too good. Real links have much higher latencies.

### Let's increases latency and jitter for the `localhost` link
Let's try:
* Latency = 100 ms.
* Jitter = 100 ms.
* Correlation between RTTs = 0.25 (<1.0).
* Statistical distribution of the RTTs = normal.

In [ ]:
# Remember to add "your_username_here ALL=(ALL) NOPASSWD: ALL" to the end of /etc/sudoers.
# Notice that these times express RTTs, not simple one-way lantencies.
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root handle 1: netem delay 20ms 5ms 25% distribution normal
!tc qdisc show dev lo

## Let's see ...

In [ ]:
!ping -c 10 localhost

OK, as we can see, at least 20 ms of latency is added to the sent packets and also to the received packets. Total, approx: 40 ms.

### Let's listen again to minimal

In [ ]:
!python ~/intercom/minimal.py --show_stats

Quite bad :-/ The chunks don't arrive with a constant cadence to the receiver (see the previous figure).

### Let's listen to the version of InterCom that uses a buffer of 100 ms

In [ ]:
!python ~/intercom/buffer.py --show_stats # 100 ms by default

Much better!

### (Optional) Remove tc rules
They will be created after, again ...

In [ ]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

## 3. [Compression](https://tecnologias-multimedia.github.io/study_guide/compress/) using [DEFLATE](https://en.wikipedia.org/wiki/DEFLATE)
We have "hidden" the jitter, but ... what happens if the available transmission bandwidth (the so called [network throughput](https://en.wikipedia.org/wiki/Throughput)) is not able to send and recive 1.41 Mbps?

A solution can be compression. The chunks can be compressed with [LZSS](https://en.wikipedia.org/wiki/Lempel-Ziv-Storer-Szymanski) (that is based on [LZ77](https://github.com/vicente-gonzalez-ruiz/LZ77)) and [Huffman Coding](https://en.wikipedia.org/wiki/Huffman_coding) (see also [this](https://vicente-gonzalez-ruiz.github.io/Huffman_coding/)).

We have choosen this text compression codec because:
1. It's fast.
2. Works well when repeated strings are found at the input.
3. I's available with [The Standard Python Library](https://docs.python.org/3/library/) ([zlib](https://docs.python.org/3/library/zlib.html)).

### Estimation of the throughput of the `localhost` link
Notice that this measurement depends heavely on the packet size, that using ping is limited to 64 KB. For this reason, the results can be only approximated :-/

In [ ]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo
!ping -c 1 -s 65527 localhost # Header length: 9 bytes

In [ ]:
!printf "Gbps = "
!echo 65527*8/0.147/2/1000/1000 | bc -l

Too high! We need harder conditions for testing InterCom ...

### Let's limit the transmission bit-rate to 300 kbps
And also set the previous latency (20 ms) and jitter (5 ms).

In [ ]:
!sudo tc qdisc add dev lo root handle 1: netem delay 20ms 5ms 25% distribution normal
!sudo tc qdisc add dev lo parent 1:1 handle 10: tbf rate 300kbit burst 1024kbit limit 1024kbit

In [ ]:
!ping -c 1 -s 65507 localhost

In [ ]:
print("Throughput =", 65507*8/522/2, "kbps")

Remember ... it's only a (quite bad) estimation.

### (Optional) Remove the previous `tc` rules

In [ ]:
!sudo tc qdisc del dev lo parent 1:1 handle 10:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

### Let's try again InterCom, but now with a limited throughput in `localhost`

In [ ]:
!python ~/intercom/buffer.py --show_stats

Notice that some chunks are lost. DEFLATE may not be enought!

### Delete `tc` rules

In [ ]:
!sudo tc qdisc del dev lo parent 1:1 handle 10:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

## 4. [Bit-rate "control"](https://tecnologias-multimedia.github.io/study_guide/quantization/) through [quantization](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb)

Quantization removes the less relevant information (mainly [electronic noise](https://en.wikipedia.org/wiki/Noise_(electronics)) ...) and helps to increase the [compression ratio](https://en.wikipedia.org/wiki/Compression_ratio). In lossy signal compression, [dead-zone quantizers](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb) are commonly used because they tend to generate 0's that like to entropy compressors.

In InterCom, we "control" the bit-rate in a lossely way because real-time bit-rate control through the quantization step $\Delta$ is computationally intensive (we must determine the [Rate/Distortion curve](https://en.wikipedia.org/wiki/Rate%E2%80%93distortion_theory) of the current chunk to find $\Delta$ before to quantize, compress and send it). The current implementation estimates the number of lost chunks per second and use this information to increase or decrease the quantization step for the chunks of the next second.

### Adapting to the current throughput

In [ ]:
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root tbf rate 200kbit burst 1024kbit limit 1024kbit
!tc qdisc show dev lo

In [ ]:
!python ~/intercom/br_control.py --show_stats

## 5. [Spatial (inter-channel) decorrelation]((https://tecnologias-multimedia.github.io/study_guide/spatial_decorrelation/))
The samples of a (stero) frame tend to have similar amplitudes. For this reason, we apply [Mid/Side stereo coding](https://en.wikipedia.org/wiki/Joint_encoding#M/S_stereo_coding) (before quantization).

In [ ]:
!python ~/intercom/intra_frame_decorrelation.py -b 100 --show_stats

## 6. Temporal (intra-channel) decorrelation
The samples of a channel exhibit temporal redundancy. Therefore, we use a [Discrete Wavelet Transform (DWT)](https://tecnologias-multimedia.github.io/study_guide/temporal_decorrelation/) to exploit it (before quantization).

In [ ]:
!python ~/my-intercom/inter_channel_decorrelation.py --show_stats

## 7. Work to do
1. Currently, quantization minimizes the MSE (Mean Square Error). Perceptual quantization can increase the compression ratio without increasing the perceived distortion.
2. Transmit video.